In [1]:
from __future__ import division
from scipy import stats
from scipy.stats.stats import pearsonr, spearmanr
import scikits.bootstrap as boot
from scipy.stats import ttest_rel
import itertools
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/')
colors=["#0000B8",'#005CE6',"#89a0ff","#990000" ,'#FF1919', "#ea8989"]
clist=[blues[-4], blues[-3], reds[-4], reds[-2]]

In [2]:
import myhddm

In [54]:
h = np.array([msm.vHH.values, msm.vFH.values, msm.vNH.values]).mean(axis=0)
f = np.array([msm.vHF.values, msm.vFF.values, msm.vNF.values]).mean(axis=0)

In [71]:
c = np.array([msm.zH2a.values, msm.zF2a.values]).mean(axis=0)
cbold = np.array([wpk_ffa.fcF, wpk_ffa.hcF, wpk_ffa.ncF]).mean(axis=0)

In [73]:
pearsonr(h, f)
pearsonr(msm.zF, msm.zH)

(0.76339295422054243, 0.00014309218186466424)

In [63]:
pearsonr(wpk_ffa.fc, wpk_ffa.hc)

(0.91720062629150023, 3.3078862988279831e-08)

In [72]:
pearsonr(c, cbold)

(0.35967809232625264, 0.13040355987506752)

In [26]:
pearsonr(msm.vF -msm.vFH, msm.zF2a-msm.zH2a)

(-0.24716470226798226, 0.30765014531911183)

In [3]:
#ewma = pd.read_csv("HDDM/Final/allsx_ewma.csv")
#allsx = pd.read_csv("fMRI/behav/all_sx.csv")
#allcor = pd.read_csv("fMRI/behav/all_sx_cor.csv")

In [4]:
b=colors[:3]
r=colors[3:]

In [86]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/')
diff = pd.read_csv('diff_rho_df.csv')

## Speed Models: Params and Norm Functions

In [11]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxPARAM')

#MSM Model RhoDF (named rho_df in previous notebooks)
msm=pd.read_csv("msm_rho_df.csv", index_col='idx')

#SpeedSimple, Speed, and MSMS models RhoDF
spds=pd.read_csv("speed_simple_rho_df.csv")
spd=pd.read_csv("speed_model_rho_df.csv")
msms=pd.read_csv("msms_rho_df.csv")

In [9]:
#Normalizing functions for all speed models
spds_norma=lambda x: x/(spds.a-(spds.a * spds.z))
spds_normb=lambda x: x/(spds.a * spds.z)

spd_norma=lambda x: x/(spd.a-(spd.a * spd.z))
spd_normb=lambda x: x/(spd.a * spd.z)
#spd_normF2A=lambda x: x/(spd.a-(spd.a * spd.z))
#spd_normH2A=lambda x: x/(spd.a * spd.z)

msms_norma=lambda x: x/(msms.a-(msms.a * msms.zF))
msms_normb=lambda x: x/(msms.a * msms.zH)

## WIN, BW, & WSS Peaks for FFA and PPA Regions

In [2]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxPK')
#WIN Peaks
wpk_ppa=pd.read_csv("ppa_rho_df.csv", index_col='idx')
wpk_ffa=pd.read_csv("ffa_rho_df.csv", index_col='idx')

#BW Peaks
bpk_ppa=pd.read_csv("ppa_bw_rhodf.csv", index_col='idx')
bpk_ffa=pd.read_csv("ffa_bw_rhodf.csv", index_col='idx')

#WSS Peaks
spk_ppa=pd.read_csv("ppa_wss_rhodf.csv", index_col='idx')
spk_ffa=pd.read_csv("ffa_wss_rhodf.csv", index_col='idx')

## WIN & WSS Min for FFA and PPA Regions

In [5]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxLO')

#WIN Minimum of TC
wlo_ppa=pd.read_csv("win_ppalow_rhodf.csv", index_col='idx')
wlo_ffa=pd.read_csv("win_ffalow_rhodf.csv", index_col='idx')

#WSS Minimum of TC
slo_ppa=pd.read_csv("wss_ppalow_rhodf.csv", index_col='idx')
slo_ffa=pd.read_csv("wss_ffalow_rhodf.csv", index_col='idx')

## WIN, BW, & WSS timecourses for FFA and PPA Regions

In [5]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxTC')

#WIN Timecourses
wtc_ffa = pd.read_csv('WIN_AllSx_FaceClusters.csv')
wtc_ppa = pd.read_csv('WIN_AllSx_HouseClusters.csv')

#BW Timecourses
btc_ffa = pd.read_csv('BW_AllSx_FaceClusters.csv')
btc_ppa = pd.read_csv('BW_AllSx_HouseClusters.csv')

#WSS Timecourses
stc_ffa = pd.read_csv('WSS_AllSx_FaceClusters.csv')
stc_ppa = pd.read_csv('WSS_AllSx_HouseClusters.csv')

## Calculate Slope and Regression Functions

In [16]:
from matplotlib import rc
rc('text', usetex=False)
'$\/CI_{95%s}[%.2f, %.2f]$' % ('\%', cis[0], cis[1])

In [6]:
def get_slopes(df, start='2', direction='pos'):

      slopes=[]

      if 'idx' in df.columns:
            sub_id_string = 'idx'
      elif 'sx' in df.columns:
            sub_id_string = 'sx'

      for i, (sx, sxvec) in enumerate(df.groupby(sub_id_string)):
            
            xhigh=np.where(sxvec.values==sxvec[start].values)[1][0]
            yhigh=sxvec[start].values[0]

            if direction=='pos':
                  xlow=np.where(sxvec.values==sxvec[start].values)[1][0]
                  ylow=sxvec[start].values[0]
                  xhigh=np.where(sxvec.values==np.max(sxvec.ix[:,start:].values[0][1:]))[1][0]
                  yhigh=np.max(sxvec.ix[:,start:].values)

            elif direction=='neg':
                  xlow=np.where(sxvec.values==np.min(sxvec.ix[:,start:].values[0][1:]))[1][0]
                  ylow=np.min(sxvec.ix[:,start:].values)
                  xhigh=np.where(sxvec.values==sxvec[start].values)[1][0]
                  yhigh=sxvec[start].values[0]
            else: 
                  print "direction must be string, 'pos' or 'neg'"
                  return []

            slopes.append((yhigh-ylow)/(xhigh-xlow))
      
      return np.array(slopes)    


def corrplots(x, y, brain, param, stim, cond, test='pearson', start='1', direct='pos', normalize=False, calc_slope=True):
      conditions = [' '.join(stimcond) for stimcond in itertools.product(stim, cond)]
      clist=[blues[-4], blues[-3], reds[-4], reds[-2]]
      f, (ax1, ax2) = plt.subplots(1,2, figsize=(9,6), sharey=True, sharex=True)

      for i, (xi, yi) in enumerate(zip(x, y)):

            if i <= 1:
                  ax=ax1
            else: 
                  ax=ax2

            if calc_slope:
                  yi = get_slopes(yi, start, direction=direct)
            if normalize:
                  xi = (xi - xi.mean())/xi.std()
                  yi = (yi - yi.mean())/yi.std()
                  
            plot_regression(xi, yi, robust=False, color=clist[::-1][i], ax=ax, cond=conditions[i], test=test)

      for i, ax in enumerate([ax1, ax2]):
            ax.legend(loc=2, fontsize=13)
            ax.set_xlabel(param, fontsize=18)
            #ax.set_title(stim[i])
      ax1.set_ylabel(brain, fontsize=18)
      plt.tight_layout()
      return f
    
def plot_regression(x, y, robust=True, color='Navy', cond=None, title=None, test='pearson', ax=None):
      if ax is None:
            f, ax = plt.subplots(1)

      if test == 'ci':
            cis = boot.ci((x, y), statfunction=stats.linregress, alpha=.05, method='bca')[:,2]
            legend = '$\/CI_{95%s}[%.2f, %.2f]$' % ('\%', cis[0], cis[1])
      elif test in ['pearson', 'spearman']:
      
            if test=='pearson':       
                  rho, pval = pearsonr(x, y)
                  rho_id =';\/r_{p}='
            elif test=='spearman':  
                  rho, pval = spearmanr(x, y) 
                  rho_id =';\/r_{s}='
                  if pval<=.05: 
                        sym = '\star'   
                  #elif pval<=.1: sym = '\dagger'
                  else: 
                        sym = 'n.s.'
                        legend =r'$%s%.2f%s$' % (rho_id, rho, sym)
      else:
            raise ValueError('test must be ci, pearson, or spearman')
      
      if cond:
            legend = cond + legend
      sns.regplot(x, y, robust=robust, fit_reg=True, line_kws={"color":color, 'lw':2, 'label':legend}, 
                scatter_kws={'color':color, 'alpha':.6}, ax=ax)
      sns.despine()
      plt.tight_layout()

# T-Test Fast, Slow Slopes

In [7]:
ppay=[wtc_ppa.query('cond=="hchouse"'), wtc_ppa.query('cond=="fchouse"'), 
   wtc_ppa.query('cond=="fcface"'), wtc_ppa.query('cond=="hcface"')]

ffay=[wtc_ffa.query('cond=="hchouse"'), wtc_ffa.query('cond=="fchouse"'), 
   wtc_ffa.query('cond=="fcface"'), wtc_ffa.query('cond=="hcface"')]

for i, d in enumerate([ppay, ffay]):
    print ttest_rel(get_slopes(d[0]), get_slopes(d[1]))
    print ttest_rel(get_slopes(d[2]), get_slopes(d[3]))

(-2.7890620934029724, 0.012117300811787893)
(1.6427290427138803, 0.11779303065327808)
(0.98828696688222506, 0.3361117920044554)
(-2.024473310270904, 0.058019215394976772)


In [9]:
ppay=[stc_ppa.query('cond=="HFast"'), stc_ppa.query('cond=="HSlow"'), 
   stc_ppa.query('cond=="FFast"'), stc_ppa.query('cond=="FSlow"')]
ffay=[stc_ffa.query('cond=="HFast"'), stc_ffa.query('cond=="HSlow"'), 
   stc_ffa.query('cond=="FFast"'), stc_ffa.query('cond=="FSlow"')]

for i, d in enumerate([ppay, ffay]):    
    print ttest_rel(get_slopes(d[0]), get_slopes(d[1]))
    print ttest_rel(get_slopes(d[2]), get_slopes(d[3]))

(2.6020431984881656, 0.018021784152901177)
(-0.25174767140368065, 0.80408796668406524)
(0.3719512218020577, 0.7142718380848434)
(7.5854414997653725, 5.1847147438776137e-07)


In [10]:
ppay=[wpk_ppa["hc-hcH"], wpk_ppa["fc-fcH"], 
    wpk_ppa["fc-fcF"], wpk_ppa["hc-hcF"]]

ffay=[wpk_ffa["hc-hcH"], wpk_ffa["fc-fcH"], 
    wpk_ffa["fc-fcF"],wpk_ffa["hc-hcF"]]

for i, d in enumerate([ppay, ffay]):    
    print ttest_rel(d[0], d[1])
    print ttest_rel(d[2], d[3])

(-2.7022402059562487, 0.014582115761102363)
(-1.0763078057320088, 0.29600461585608095)
(0.89884650297385538, 0.3806014947047166)
(-3.8823093691189894, 0.0010916141767539893)


In [11]:
ppay=[wpk_ppa["hcH"], wpk_ppa["fcH"], 
    wpk_ppa["fcF"], wpk_ppa["hcF"]]

ffay=[wpk_ffa["hcH"], wpk_ffa["fcH"], 
    wpk_ffa["fcF"], wpk_ffa["hcF"]]

for i, d in enumerate([ppay, ffay]):    
    print ttest_rel(d[0], d[1])
    print ttest_rel(d[2], d[3])

(-3.1517337984922698, 0.0055173157118805771)
(-1.4262617459612432, 0.17090713143942196)
(-0.70091747359790535, 0.49231689341504747)
(-2.5256425319695994, 0.021148387293976834)


In [12]:
ppay=[wpk_ppa["hc"], wpk_ppa["fc"], 
    wpk_ppa["fc"], wpk_ppa["hc"]]

ffay=[wpk_ffa["hc"], wpk_ffa["fcH"], 
    wpk_ffa["fcF"], wpk_ffa["hcF"]]

for i, d in enumerate([ppay, ffay]):    
    print ttest_rel(d[0], d[1])
    print ttest_rel(d[2], d[3])

(-0.041906928813351216, 0.96703425001621868)
(0.041906928813351216, 0.96703425001621868)
(-1.0439516008872276, 0.31032619980940035)
(-2.5256425319695994, 0.021148387293976834)


In [13]:
brain = ['PPA Slope', 'FFA Slope']
param = 'Drift-Rate'
stim = ['House','Face']
cond = ['Valid', 'Invalid']

x=[msm.vHH_n_zH2b.values, msm.vFH_n_zF2b.values, 
   msm.vFF_n_zH2b.values, msm.vHF_n_zF2b.values]

ppay=[wtc_ppa.query('cond=="hchouse"'), wtc_ppa.query('cond=="fchouse"'), 
   wtc_ppa.query('cond=="fcface"'), wtc_ppa.query('cond=="hcface"')]

ffay=[wtc_ffa.query('cond=="hchouse"'), wtc_ffa.query('cond=="fchouse"'), 
   wtc_ffa.query('cond=="fcface"'), wtc_ffa.query('cond=="hcface"')]

for i, data in enumerate([ppay, ffay]):
    region=brain[i]
    f = corrplots(x, data, region, param, stim, cond, direct='pos', start='1', 
                  normalize=True, test='pearson', calc_slope=True)
    
    txt = f.suptitle('$WIN\/GLM:\/%s\/Slope\/and\/Valid/Invalid\/Drift-Rates$' % brain[i][:3], y=.92, fontsize=18)
    plt.subplots_adjust(top=.86)
    #f.savefig('/Users/kyle/Desktop/WIN_SlopeDrift_%s.png' % brain[i], dpi=300)

In [12]:
brain = ['PPA Slope', 'FFA Slope']
param = 'Drift-Rate'
stim = ['House','Face']
cond = ['Fast', 'Slow']

x=[spd_normb(spd.vHFast.values), spd_normb(spd.vHSlow.values), 
   spd_norma(spd.vFFast.values), spd_norma(spd.vFSlow.values)]

#x=[spd_normb(msms.vHFast.values), spd_normb(msms.vHSlow.values), 
#   spd_norma(msms.vFFast.values), spd_norma(msms.vFSlow.values)]


ppay=[stc_ppa.query('cond=="HFast"'), stc_ppa.query('cond=="HSlow"'), 
   stc_ppa.query('cond=="FFast"'), stc_ppa.query('cond=="FSlow"')]
ffay=[stc_ffa.query('cond=="HFast"'), stc_ffa.query('cond=="HSlow"'), 
   stc_ffa.query('cond=="FFast"'), stc_ffa.query('cond=="FSlow"')]

for i, data in enumerate([ppay, ffay]):
      region=brain[i]
      f = corrplots(x, data, region, param, stim, cond, start='1', 
                  test='pearson', normalize=True, calc_slope=True)

      txt = f.suptitle('$WSS/GLM:\/%s\/Slope\/and\/Fast/Slow\/Drift-Rates$' % brain[i][:3], y=.92, fontsize=18)
      plt.subplots_adjust(top=.86)
      #f.savefig('/Users/kyle/Desktop/WSS_SlopeDrift_%s.png' % brain[i], dpi=300)

UnboundLocalError: local variable 'legend' referenced before assignment

In [17]:
brain = ['PPA Slope', 'FFA Slope']
param = 'Drift-Rate'
stim = ['House','Face']
cond = ['Fast', 'Slow']

x=[spd_normb(spd.vHFast.values), spd_normb(spd.vHSlow.values), 
   spd_norma(spd.vFFast.values), spd_norma(spd.vFSlow.values)]
x=[spd.vHFast.values, spd.vHSlow.values, 
    spd.vFFast.values, spd.vFSlow.values]
#x=[spd.z2a.values, spd.z2b.values, 
#    spd.z2a.values, spd.z2a.values]

ppay=[stc_ppa.query('cond=="HFast"'), stc_ppa.query('cond=="HSlow"'), 
   stc_ppa.query('cond=="FFast"'), stc_ppa.query('cond=="FSlow"')]
ffay=[stc_ffa.query('cond=="HFast"'), stc_ffa.query('cond=="HSlow"'), 
   stc_ffa.query('cond=="FFast"'), stc_ffa.query('cond=="FSlow"')]

for i, data in enumerate([ppay, ffay]):
    region=brain[i]
    f = corrplots(x, data, region, param, stim, cond, start='2', 
                  test='pearson', normalize=True, direct='pos', calc_slope=True)
    
    txt = f.suptitle('$WSS/GLM:\/%s\/Slope\/and\/Fast/Slow\/Drift-Rates$' % brain[i][:3], y=.92, fontsize=18)
    plt.subplots_adjust(top=.86)
    #f.savefig('/Users/kyle/Desktop/WSS_SlopeDrift_%s.png' % brain[i], dpi=300)

In [20]:
brain = ['PPA', 'FFA']
param = 'Distance to Threshold'
stim = ['House','Face']
cond = ['Valid', 'Invalid']

x=[msm.zH2b.values, msm.zF2b.values, 
   msm.zF2a.values, msm.zH2a.values]

ppay=[wpk_ppa["hc-hcH"], wpk_ppa["fc-fcH"], 
    wpk_ppa["fc-fcF"],wpk_ppa["hc-hcF"]]

ffay=[wpk_ffa["hc-hcH"], wpk_ffa["fc-fcH"], 
    wpk_ffa["fc-fcF"],wpk_ffa["hc-hcF"]]

for i, data in enumerate([ppay, ffay]):
    f = corrplots(x, data, brain, param, stim, cond, 
              normalize=True, test='pearson', calc_slope=False)

    f.suptitle('$WIN\/GLM:%s\/\Delta_{CueStim}\/\/and\/\/\Delta_{Bound}$'% brain[i], y=.92, fontsize=18)
    plt.subplots_adjust(top=.86)
    f.savefig('/Users/kyle/Desktop/WIN_%s_DeltaCueStim_DeltaThreshold.png'%brain[i], dpi=300)

In [42]:
brain = ['PPA', 'FFA']
param = 'Distance to Threshold'
stim = ['House','Face']
cond = ['Valid', 'Invalid']

x=[msm.zH2b.values, msm.zF2b.values, 
   msm.zF2a.values, msm.zH2a.values]

ppay=[wpk_ppa["hc-hcH"], wpk_ppa["fc-fcH"], 
    wpk_ppa["fc-fcF"],wpk_ppa["hc-hcF"]]

ffay=[wpk_ffa["hc-hcH"], wpk_ffa["fc-fcH"], 
    wpk_ffa["fc-fcF"],wpk_ffa["hc-hcF"]]

for i, data in enumerate([ppay, ffay]):
    f = corrplots(x, data, brain, param, stim, cond, 
              normalize=True, test='pearson', calc_slope=False)

    f.suptitle('$WIN\/GLM:%s\/\Delta_{CueStim}\/\/and\/\/\Delta_{Bound}$'%brain[i], y=.92, fontsize=18)
    plt.subplots_adjust(top=.86)
    f.savefig('/Users/kyle/Desktop/WIN_%s_DeltaCueStim_DeltaThreshold.png'%brain[i], dpi=300)

In [43]:
brain = ['PPA', 'FFA']
param = 'Distance to Threshold'
stim = ['House','Face']
cond = ['Valid', 'Invalid']

x=[msm.zH2b.values, msm.zF2b.values, 
   msm.zF2a.values, msm.zH2a.values]

ppay=[bpk_ppa["hc-hcH"], bpk_ppa["fc-fcH"], 
    bpk_ppa["fc-fcF"],bpk_ppa["hc-hcF"]]

ffay=[bpk_ffa["hc-hcH"], bpk_ffa["fc-fcH"], 
    bpk_ffa["fc-fcF"],bpk_ffa["hc-hcF"]]

for i, data in enumerate([ppay, ffay]):
    f = corrplots(x, data, brain, param, stim, cond, 
              normalize=True, test='pearson', calc_slope=False)

    f.suptitle('$BW\/GLM:%s\/\Delta_{CueStim}\/\/and\/\/\Delta_{Bound}$'%brain[i], y=.92, fontsize=18)
    plt.subplots_adjust(top=.86)
    f.savefig('/Users/kyle/Desktop/BW_%s_DeltaCueStim_DeltaThreshold.png'%brain[i], dpi=300)

In [187]:
wtc_ffa['low']=stc_ffa.min(axis=1)
wtc_ppa['low']=stc_ppa.min(axis=1)
wlo_ppa = wtc_ppa.pivot(index='sx', columns='cond', values='low')
wlo_ffa = wtc_ffa.pivot(index='sx', columns='cond', values='low')
#stc_drop['sx']
#stc_ppa.groupby('cond').min().T

In [214]:
spd['z2a']=spd['a']-(spd['z']*spd['a'])
spd['z2b']=spd['z']*spd['a']
spd['vFFast_n_z2a']=spd['vFFast']/spd['z2a']
spd['vHFast_n_z2b']=spd['vHFast']/spd['z2b']
spd['vFSlow_n_z2a']=spd['vFSlow']/spd['z2a']
spd['vHSlow_n_z2b']=spd['vHSlow']/spd['z2b']



In [155]:
spd['z']=np.array([ 0.58806424,  0.58886978,  0.59901976,  0.65960359,  0.69008232,
        0.6407637 ,  0.64188813,  0.66934457,  0.65173494,  0.62666071,
        0.54893321,  0.45613744,  0.67907129,  0.63864626,  0.58237093,
        0.70798236,  0.67800684,  0.66896503,  0.71004195])